<a href="https://colab.research.google.com/github/Nonglassy/Gazua/blob/main/%EA%B2%BD%EC%A0%9C%EC%A7%80%ED%91%9C_%EA%B8%B0%EC%88%A0%EC%A7%80%ED%91%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0(%EB%88%84%EB%A5%B4%EB%8A%94_%EC%88%9C%EA%B0%84_%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#경제지표
import requests
import pandas as pd
import json

def getdata(search_text,st_date,end_date):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36','X-Requested-With':'XMLHttpRequest',}
    data={'search_text':search_text,}
    result1=requests.post('https://www.investing.com/search/service/searchTopBar',data=data,headers=headers)
    data=result1.json()
    first_quote_result=data['quotes'][0]
    curr_id=first_quote_result['pairId'] 
    data={'curr_id':curr_id,'st_date':st_date,'end_date':end_date,'interval_sec':'Daily','action':'historical_data',}
    result2=requests.post('https://www.investing.com/instruments/HistoricalDataAjax',data=data,headers=headers)
    data=result2.text
    data=pd.read_html(data)
    hist_data=data[0]
    return pd.DataFrame(hist_data)


In [45]:
from datetime import datetime
today=datetime.today().strftime('%Y-%m-%d')
now=datetime.strptime(today, "%Y-%m-%d").strftime('%m/%d/%Y')

Bitcoin=getdata('bitcoin','01/01/2018',now)
TenYear=getdata('united states 10-Year','01/01/2018',now)
OVX=getdata('ovx','01/01/2018',now)
VIX=getdata('vix','01/01/2018',now)

In [47]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

lr=LinearRegression()
imp=IterativeImputer(estimator=lr, tol=1e-10, max_iter=30, verbose=2, imputation_order='roman')

Bitcoin=Bitcoin.drop(Bitcoin.columns[1:],axis=1)
TenYear=TenYear.drop(TenYear.columns[5],axis=1)
OVX=OVX.drop(OVX.columns[5:7],axis=1)
VIX=VIX.drop(VIX.columns[5:7],axis=1)
TenYear.columns=['Date','YOpen','YHigh','YLow','YClose']
OVX.columns=['Date','ovxOpen','ovxHigh','ovxYLow','ovxClose']
VIX.columns=['Date','vixOpen','vixHigh','vixLow','vixClose']

data=pd.merge(left=Bitcoin, right=TenYear, how='outer',on="Date")
data=pd.merge(left=data, right=OVX, how='outer',on="Date")
data=pd.merge(left=data, right=VIX, how='outer',on="Date")
data=data.drop(data.columns[0],axis=1)

data=imp.fit_transform(data)

from datetime import datetime
from pytz import timezone

Otherdata = pd.DataFrame(data, columns=['YOpen','YHigh','YLow','YClose','ovxOpen','ovxHigh','ovxLow','ovxClose','vixOpen','vixHigh','vixLow','vixClose'])
Otherdata=pd.concat([Bitcoin, Otherdata], axis = 1)
Otherdata['Date']=pd.to_datetime(Otherdata['Date'])
Otherdata=Otherdata.sort_values(by=['Date'])
Otherdata=Otherdata.drop(Otherdata.columns[0],axis=1)
Otherdata.reset_index(drop=True, inplace = True)

[IterativeImputer] Completing matrix with shape (1708, 12)
[IterativeImputer] Ending imputation round 1/30, elapsed time 0.05
[IterativeImputer] Change: 9.652572060702958, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 2/30, elapsed time 0.08
[IterativeImputer] Change: 3.519314168625563, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 3/30, elapsed time 0.11
[IterativeImputer] Change: 4.308137938446492, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 4/30, elapsed time 0.14
[IterativeImputer] Change: 4.296337766712618, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 5/30, elapsed time 0.17
[IterativeImputer] Change: 4.136018167552088, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer] Ending imputation round 6/30, elapsed time 0.22
[IterativeImputer] Change: 3.953660080154606, scaled tolerance: 5.171900000000001e-08 
[IterativeImputer

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [8]:
#기술지표
!pip install pyupbit
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 7.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3959    0  3959    0     0  19894      0 --:--:-- --:--:-- --:--:-- 19894
100  503k  100  503k    0     0  1012k      0 --:--:-- --:--:-- --:--:-- 1012k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3987    0  3987    0     0  27308      0 --:--:-- --:--:-- --:--:-- 27122
100  406k  100  406k    0     0  1021k      0 --:--:-- --:--:-- --:--:-- 1021k


In [20]:
import pyupbit
import pandas as pd
import numpy as np
import talib as ta

def getPrice(search,n,finish_date):
    df=pyupbit.get_ohlcv(search,'hour',p,to=finish_date)
    df=df.drop("value",axis=1)
    return pd.DataFrame(df)

def getADX(data):
    adx=ta.ADX(data['high'],data['low'],data['close'],timeperiod=14)
    adx=adx[50:]
    return pd.DataFrame(adx)

def getMA50(data):
    MA50=ta.MA(data['close'],timeperiod=50, matype=0)
    MA50=MA50[50:]
    return pd.DataFrame(MA50)

def getMACD_signal(data):
    MACD,MACD_signal,MACD_hist=ta.MACD(data['close'],fastperiod=12, slowperiod=26, signalperiod=9)
    MACD_signal=MACD_signal[50:]
    return pd.DataFrame(MACD_signal)

today=datetime.now().date()
start=datetime.strptime("20171111", "%Y%m%d").date()
p=(today-start).days

BTC=getPrice('KRW-BTC',p,'today')
ETH=getPrice('KRW-ETH',p,'today')
ETC=getPrice('KRW-ETC',p,'today')
ADA=getPrice('KRW-ADA',p,'today')
QTUM=getPrice('KRW-QTUM',p,'today')
XRP=getPrice('KRW-XRP',p,'today')

ADX_BTC=getADX(BTC)
ADX_ETH=getADX(ETH)
ADX_ETC=getADX(ETC)
ADX_ADA=getADX(ADA)
ADX_QTUM=getADX(QTUM)
ADX_XRP=getADX(XRP)

MA50_BTC=getMA50(BTC)
MA50_ETH=getMA50(ETH)
MA50_ETC=getMA50(ETC)
MA50_ADA=getMA50(ADA)
MA50_QTUM=getMA50(QTUM)
MA50_XRP=getMA50(XRP)


MACD_signal_BTC=getMACD_signal(BTC)
MACD_signal_ETH=getMACD_signal(ETH)
MACD_signal_ETC=getMACD_signal(ETC)
MACD_signal_ADA=getMACD_signal(ADA)
MACD_signal_QTUM=getMACD_signal(QTUM)
MACD_signal_XRP=getMACD_signal(XRP)


In [21]:
Fin_BTC=pd.concat([ADX_BTC,MA50_BTC,MACD_signal_BTC],axis=1)
Fin_BTC.columns=['ADX_BTC','MA50_BTC','MACD_signal_BTC']
Fin_ETH=pd.concat([ADX_ETH,MA50_ETH,MACD_signal_ETH],axis=1)
Fin_ETH.columns=['ADX_ETH','MA50_ETH','MACD_signal_ETH']
Fin_ETC=pd.concat([ADX_ETC,MA50_ETC,MACD_signal_ETC],axis=1)
Fin_ETC.columns=['ADX_ETC','MA50_ETC','MACD_signal_ETC']
Fin_ADA=pd.concat([ADX_ADA,MA50_ADA,MACD_signal_ADA],axis=1)
Fin_ADA.columns=['ADX_ADA','MA50_ADA','MACD_signal_ADA']
Fin_QTUM=pd.concat([ADX_QTUM,MA50_QTUM,MACD_signal_QTUM],axis=1)
Fin_QTUM.columns=['ADX_QTUM','MA50_QTUM','MACD_signal_QTUM']
Fin_XRP=pd.concat([ADX_XRP,MA50_XRP,MACD_signal_XRP],axis=1)
Fin_XRP.columns=['ADX_XRP','MA50_XRP','MACD_signal_XRP']

Fin_Last=pd.concat([Fin_BTC,Fin_ETH,Fin_ETC,Fin_ADA,Fin_QTUM,Fin_XRP],axis=1)
Fin_Last.reset_index(drop=True, inplace = True)
print(Fin_Last)

        ADX_BTC    MA50_BTC  MACD_signal_BTC    ADX_ETH   MA50_ETH  \
0     25.704880  15635440.0     1.671972e+06  20.193102   677713.0   
1     25.115793  15890960.0     1.529126e+06  20.874206   694024.0   
2     24.568783  16157140.0     1.417729e+06  21.840361   712234.0   
3     25.034719  16458180.0     1.368570e+06  23.694196   733436.0   
4     26.797373  16832860.0     1.422367e+06  25.885890   757222.0   
...         ...         ...              ...        ...        ...   
1703  26.289061  29890820.0    -5.350952e+05  24.641658  2174130.0   
1704  26.809863  29911320.0    -5.979861e+05  23.914416  2188300.0   
1705  26.864642  29920040.0    -6.476737e+05  22.526020  2200400.0   
1706  26.944028  29918620.0    -6.869831e+05  21.368137  2210900.0   
1707  27.116829  29908960.0    -7.173259e+05  20.292959  2218310.0   

      MACD_signal_ETH    ADX_ETC  MA50_ETC  MACD_signal_ETC    ADX_ADA  \
0       119501.776740  27.858373   32466.0      2947.344014  54.961642   
1       118

In [52]:
#LSTM 모델에 넣을 마지막

Data=pd.concat([Otherdata,Fin_Last],axis=1)
data=Data[-3:]
print(Data.isnull().sum())
print(data)

YOpen               0
YHigh               0
YLow                0
YClose              0
ovxOpen             0
ovxHigh             0
ovxLow              0
ovxClose            0
vixOpen             0
vixHigh             0
vixLow              0
vixClose            0
ADX_BTC             0
MA50_BTC            0
MACD_signal_BTC     0
ADX_ETH             0
MA50_ETH            0
MACD_signal_ETH     0
ADX_ETC             0
MA50_ETC            0
MACD_signal_ETC     0
ADX_ADA             0
MA50_ADA            0
MACD_signal_ADA     0
ADX_QTUM            0
MA50_QTUM           0
MACD_signal_QTUM    0
ADX_XRP             0
MA50_XRP            0
MACD_signal_XRP     0
dtype: int64
        YOpen     YHigh      YLow    YClose    ovxOpen    ovxHigh     ovxLow  \
1705  3.19500  3.257000  3.286000  3.191000  53.900000  54.550000  54.590000   
1706  1.95288  1.951896  1.981155  1.922532  43.075124  42.964299  47.198815   
1707  1.95288  1.951896  1.981155  1.922532  43.075124  42.964299  47.198815   

      